Donau-Ries (Lkr): Äpfel und Birnen

Der Datensatz enthält Informationen zu besonderen Apfel- und Birnensorten in Nordschwaben, die im Rahmen eines LEADER-Projekts von 2016 bis 2020 erfasst wurden.
Dokumentiert wurden Sortenvielfalt, Verbreitung, Häufigkeit und Gefährdungsstatus von rund 400???(173) historischen Obstsorten in den Landkreisen
Donau-Ries, Neu-Ulm, Augsburg und Aichach-Friedberg. Ziel ist der Erhalt genetischer Vielfalt und regionaler Kultursorten durch fundierte Bestandsaufnahme und Dokumentation.

quelle : https://www.govdata.de/suche/daten/donau-ries-lkr-apfel-und-birnen

In [43]:
import requests

url = "https://fms.donau-ries.de/formcycle/datenabfragedb?mandantName=LRA-Donau-Ries&name=aepfel-und-birnen"

# Anfrage schicken
response = requests.get(url)

# prüfen ob alles ok war
if response.status_code == 200:
    data = response.json()   # JSON direkt als Python-Dict/Liste
    #print(data['result'])
else:
    print("Fehler beim Abrufen:", response.status_code)



In [44]:
from bs4 import BeautifulSoup
ls_keys = ["geschichte", "frucht", "reifezeit", "standort", "anfälligkeit", "blüte", "sonstiges" ]
alle_sorten = {}
number  = 0
for eintrag in data["result"]:
    alias = eintrag["alias"]
    
    html_text = eintrag["text_inhalt"]
    #quellen = eintrag.get("quellen", "")

    soup = BeautifulSoup(html_text, "html.parser")

    sorten_dict = {}
    current_key = None
    #print(soup)
    for tag in soup.find_all("p"):
        text = tag.get_text(strip=True)
        #print(tag)
        # Neue Sektion wenn <strong> drin ist
        if tag.find("strong"):
            current_key = text.replace(":", "").lower()
            if current_key=="früchte":
                current_key="frucht"
            if current_key=="früchte":
                current_key="frucht"
            sorten_dict[current_key] = ""
        else:
            if current_key:
                if sorten_dict[current_key]:
                    sorten_dict[current_key] += " " + text
                else:
                    sorten_dict[current_key] = text
    sorten_dict["ind"] = number
    alle_sorten[alias] = sorten_dict
    number += 1
#print(alle_sorten["blauapfel"])
#alle_sorten["horneburger-pfannkuchenapfel"]

#alle_sorten["suesskirsche-skeena"]

In [45]:

LS_KEYS = ["geschichte", "frucht", "reifezeit", "standort", "anfälligkeit", "blüte", "sonstiges", "ind"]
alle_sorten_cleaned  = {}
sorten_cleaned  = {}
for key, value in alle_sorten.items():
    
    for ke, val in value.items():
        if ke in LS_KEYS:
            sorten_cleaned[ke]= val
            continue
        else:
            for wort in LS_KEYS:
                if ke.startswith(wort):
                    sorten_cleaned[wort] =  ke[len(wort)::] 
                else:
                    continue
    alle_sorten_cleaned[key] = sorten_cleaned
    sorten_cleaned  = {}



In [46]:
import pandas as pd
df = pd.DataFrame(alle_sorten_cleaned)
df  = df.T
lss = df[df["anfälligkeit"].notna()]["anfälligkeit"].tolist()
lsn = df[df["anfälligkeit"].notna()]["ind"].tolist()

anf = dict(zip(lsn,lss))
df = df.reset_index().rename(columns={"index": "name"})


In [47]:
import re
Feuerbrand_filtered = {}
for i, s in anf.items():
    if "feuer" in anf[i].lower():
        for st in re.split(r"[,.]", anf[i]):
           if "feuer" in st.lower():
                Feuerbrand_filtered[i] = st 

In [48]:
#Feuerbrand_filtered
# gibt es einen unterschied zwischen gefährdet und anfällig?
# Wie kann man das Feuerbrandgefahr einstufen?

In [49]:
import re

LEVELS = {"sehr stark": 4, "stark": 3, "mittel": 2, "gering": 1, "resistent": 0}

def classify_level(text: str):
    if not text:
        return None
    # normalisieren
    t = text.lower()
    t = re.sub(r"[-–—]", " ", t)     # Bindestriche als Leerzeichen
    t = re.sub(r"\s+", " ", t).strip()

    # 0) resistent / nicht anfällig
    if any(x in t for x in ["feuerbrandresistent", "widerstandsfähig gegen feuerbrand", "nicht anfällig"]):
        return LEVELS["resistent"]

    # 4) sehr stark
    if ("sehr stark" in t or "besonders" in t or "hoch " in t) and ("anfällig" in t or "gefährdet" in t):
        return LEVELS["sehr stark"]

    # 3) stark
    if "stark" in t and ("anfällig" in t or "gefährdet" in t):
        return LEVELS["stark"]

    # 2) mittel / mäßig / mittelstark / mittelanfällig / mittlere(r/s)
    if any(x in t for x in ["mittelstark", "mittelanfällig", "mäßig", "mittlere ", "mittlerer", "mittleren", "mittleres", "mittlere", "mittlere", "mittlere"]) \
       or "mittlere" in t:
        return LEVELS["mittel"]

    # 1) gering / wenig / kaum
    if any(x in t for x in ["gering", "wenig", "kaum"]):
        return LEVELS["gering"]

    # nichts eindeutiges
    return None

classified = {k: classify_level(v) for k, v in Feuerbrand_filtered.items()}
#df_classified = pd.DataFrame(classified)
df_classified = pd.DataFrame(list(classified.items()), columns=["ind", "class_feuerbrand"])
df_classified.head(10)

,ind,class_feuerbrand
0,0,NaN
1,1,3.0
2,2,3.0
3,3,2.0
4,4,3.0
5,8,3.0
6,9,1.0
7,13,1.0
8,19,3.0
9,22,3.0


In [50]:
df_small = df[["name","ind"]]
merged = pd.merge(df_small, df_classified, on="ind", how="inner")
merged.head(10)

,name,ind,class_feuerbrand
0,alkmene,0,NaN
1,ananasrenette,1,3.0
2,baumanns-renette,2,3.0
3,berner-rosenapfel,3,2.0
4,bittenfelder-saemling,4,3.0
5,brettacher-gewuerzapfel,8,3.0
6,croncels,9,1.0
7,duelmener-rosenapfel,13,1.0
8,gelber-bellefleur,19,3.0
9,gewuerzluiken,22,3.0
